In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [3]:
# Top level data directory. Here we assume the format of the directory conforms
#   to the ImageFolder structure
data_dir = 'hymenoptera_data'

# Models to choose from [resnet, alexnet, vgg, squeezenet, densenet, inception]
model_name = ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']

# Number of classes in the dataset
num_classes = 2

# Batch size for training (change depending on how much memory you have)
batch_size = 8

# Number of epochs to train for
num_epochs = 15

# Flag for feature extracting. When False, we finetune the whole model,
#   when True we only update the reshaped layer params
feature_extract = True

In [4]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history


In [5]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False


In [6]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size




In [8]:
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
# Iterate through the all the models

for current_model in model_name:
    # Store model parameter in these lists
    model_fit_explain = []
    histogram_explain = []
    
    # Initialize the model for this run
    model_ft, input_size = initialize_model(current_model, num_classes, feature_extract, use_pretrained=True)
    
    # Data augmentation and normalization for training
    # Just normalization for validation
    data_transforms = {
        'train': transforms.Compose([
            transforms.RandomResizedCrop(input_size),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
        'val': transforms.Compose([
            transforms.Resize(input_size),
            transforms.CenterCrop(input_size),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ]),
    }
    
    # Create training and validation datasets
    image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
    # Create training and validation dataloaders
    dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}
    # Send the model to GPU
    model_ft = model_ft.to(device)

    # Gather the parameters to be optimized/updated in this run. If we are
    #  finetuning we will be updating all parameters. However, if we are
    #  doing feature extract method, we will only update the parameters
    #  that we have just initialized, i.e. the parameters with requires_grad
    #  is True.
    params_to_update = model_ft.parameters()
    print("Params to learn:")
    if feature_extract:
        params_to_update = []
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                params_to_update.append(param)
                print("\t",name)
    else:
        for name,param in model_ft.named_parameters():
            if param.requires_grad == True:
                print("\t",name)

    # Observe that all parameters are being optimized
    optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

    # Loss Function
    criterion = nn.CrossEntropyLoss()

    # Train and evaluate
    model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

    model_fit_explain = model_fit_explain.append(model_ft)
    histogram_explain = histogram_explain.append(hist)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to C:\Users\anark/.cache\torch\checkpoints\resnet18-5c106cde.pth



Params to learn:
	 fc.weight
	 fc.bias
Epoch 0/14
----------
train Loss: 0.5996 Acc: 0.6434
val Loss: 0.2626 Acc: 0.9412

Epoch 1/14
----------
train Loss: 0.4087 Acc: 0.8279
val Loss: 0.2272 Acc: 0.9346

Epoch 2/14
----------
train Loss: 0.3653 Acc: 0.8525
val Loss: 0.2563 Acc: 0.9150

Epoch 3/14
----------
train Loss: 0.3178 Acc: 0.8525
val Loss: 0.1743 Acc: 0.9477

Epoch 4/14
----------
train Loss: 0.2670 Acc: 0.8770
val Loss: 0.2196 Acc: 0.9281

Epoch 5/14
----------
train Loss: 0.4069 Acc: 0.8033
val Loss: 0.1765 Acc: 0.9412

Epoch 6/14
----------
train Loss: 0.3425 Acc: 0.8607
val Loss: 0.1711 Acc: 0.9608

Epoch 7/14
----------
train Loss: 0.3480 Acc: 0.8361
val Loss: 0.1760 Acc: 0.9542

Epoch 8/14
----------
train Loss: 0.4281 Acc: 0.8320
val Loss: 0.1685 Acc: 0.9412

Epoch 9/14
----------
train Loss: 0.2633 Acc: 0.8852
val Loss: 0.1775 Acc: 0.9412

Epoch 10/14
----------
train Loss: 0.2914 Acc: 0.8811
val Loss: 0.1658 Acc: 0.9477

Epoch 11/14
----------
train Loss: 0.2659 Acc:

Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to C:\Users\anark/.cache\torch\checkpoints\alexnet-owt-4df8aa71.pth



Params to learn:
	 classifier.6.weight
	 classifier.6.bias
Epoch 0/14
----------
train Loss: 0.5344 Acc: 0.7992
val Loss: 0.3447 Acc: 0.8824

Epoch 1/14
----------
train Loss: 0.3868 Acc: 0.8484
val Loss: 0.3441 Acc: 0.8824

Epoch 2/14
----------
train Loss: 0.2216 Acc: 0.9016
val Loss: 0.4111 Acc: 0.9085

Epoch 3/14
----------
train Loss: 0.3131 Acc: 0.9057
val Loss: 0.3406 Acc: 0.9020

Epoch 4/14
----------
train Loss: 0.2452 Acc: 0.9016
val Loss: 0.4617 Acc: 0.8627

Epoch 5/14
----------
train Loss: 0.2786 Acc: 0.9139
val Loss: 0.3773 Acc: 0.8954

Epoch 6/14
----------
train Loss: 0.0832 Acc: 0.9754
val Loss: 0.4598 Acc: 0.8824

Epoch 7/14
----------
train Loss: 0.1662 Acc: 0.9426
val Loss: 0.3908 Acc: 0.9085

Epoch 8/14
----------
train Loss: 0.1884 Acc: 0.9426
val Loss: 0.3462 Acc: 0.9281

Epoch 9/14
----------
train Loss: 0.1588 Acc: 0.9549
val Loss: 0.3694 Acc: 0.9085

Epoch 10/14
----------
train Loss: 0.1804 Acc: 0.9303
val Loss: 0.4150 Acc: 0.9085

Epoch 11/14
----------
tra

Downloading: "https://download.pytorch.org/models/vgg11_bn-6002323d.pth" to C:\Users\anark/.cache\torch\checkpoints\vgg11_bn-6002323d.pth


ValueError: not enough values to unpack (expected 2, got 0)

In [12]:
model_ft

SqueezeNet(
  (features): Sequential(
    (0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
    (3): Fire(
      (squeeze): Conv2d(96, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (4): Fire(
      (squeeze): Conv2d(128, 16, kernel_size=(1, 1), stride=(1, 1))
      (squeeze_activation): ReLU(inplace=True)
      (expand1x1): Conv2d(16, 64, kernel_size=(1, 1), stride=(1, 1))
      (expand1x1_activation): ReLU(inplace=True)
      (expand3x3): Conv2d(16, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (expand3x3_activation): ReLU(inplace=True)
    )
    (5): Fire(
   